cos-sync

syncs two S3 compliant COS buckets (unidirectional)

In [ ]:
!pip install kfp kfp-tekton

In [ ]:
import kfp
from kfp.components import create_component_from_func
from kfp_tekton.compiler import TektonCompiler
import kfp.dsl as dsl
import kfp.dsl as dsl
import kfp
from kfp_tekton.compiler import TektonCompiler
from kfp.compiler import Compiler
import kfp.components as comp


In [ ]:
 
util_cos = comp.load_component_from_url('https://raw.githubusercontent.com/claimed-framework/component-library/main/component-library/util/util-cos.yaml')
#util_cos = comp.load_component_from_file('util-cos.yaml') # for testing locally

In [ ]:


# Define your pipeline function
@dsl.pipeline(
    name="cos-sync",
    description="syncs two S3 compliant COS buckets (unidirectional)"
)
def agb_inference_preprocess(
        source_cos_path: str,
        source_access_key_id: str,
        source_secret_access_key: str,
        source_endpoint: str,
        source_bucket_name: str,
        target_local_path: str,
        target_access_key_id: str,
        target_secret_access_key: str,
        target_endpoint: str,
        target_bucket_name: str,
        buffer_dir: str = '/data/',
        pvc_mount_path: str = '/data/',
        pvc_name: str = 'kfp-data-pvc-z'
,
):
  
    step1 = util_cos(
            access_key_id=source_access_key_id,
            secret_access_key=source_secret_access_key,
            endpoint=source_endpoint,
            bucket_name=source_bucket_name,
            path=source_cos_path,
            recursive='True',
            operation='sync_to_local',
            target=buffer_dir,
    )

    step2 = util_cos(
            access_key_id=target_access_key_id,
            secret_access_key=target_secret_access_key,
            endpoint=target_endpoint,
            bucket_name=target_bucket_name,
            source=str(buffer_dir)+str(target_local_path),
            recursive='True',
            operation='sync_to_cos',
    )

    step2.after(step1)


    pipeline_volume = dsl.PipelineVolume(pvc=pvc_name)


    step1.add_pvolumes({pvc_mount_path: pipeline_volume})
    step2.add_pvolumes({pvc_mount_path: pipeline_volume})

TektonCompiler().compile(agb_inference_preprocess, 'cos-sync.tekton.yaml')
Compiler().compile(agb_inference_preprocess, 'cos-sync.argo.yaml')